# 최적화 전/후 불량률 비교 (XGBoost으로 예측)

In [1]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# scaling
from sklearn.preprocessing import StandardScaler
# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 로지스틱 회귀
from statsmodels.api import Logit
# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, f1_score 
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import matplotlib.pyplot as plt
# 글꼴 적용
plt.rc('font', family = 'NanumBarunGothic')
# 음수 표시
matplotlib.rc('axes', unicode_minus=False)

## 1. 데이터 구성하기 및 정제

In [3]:
df_raw = pd.read_csv("/home/piai/바탕화면/bat_process.csv")
df_raw.head()

,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an,dt_start_old,sa4_box_row,sa4_box_col,sa4_box_dan
0,LOT-100,TRAY-1023,01SEP2022:10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,6.8,4718.0,37583.0,94.0,565.0,164.0,01SEP2020:10:01:25,07열,27연,03단
1,LOT-100,TRAY-1023,01SEP2022:10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,6.8,4718.0,37574.0,94.0,565.0,73.0,01SEP2020:10:01:25,07열,27연,03단
2,LOT-100,TRAY-1023,01SEP2022:10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,6.9,4700.0,37579.0,91.0,585.0,-57.0,01SEP2020:10:01:25,07열,27연,03단
3,LOT-100,TRAY-1023,01SEP2022:10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,6.9,4688.0,37583.0,93.0,606.0,99.0,01SEP2020:10:01:25,07열,27연,03단
4,LOT-100,TRAY-1023,01SEP2022:10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,6.8,4696.0,37570.0,91.0,605.0,36.0,01SEP2020:10:01:25,07열,27연,03단


In [4]:
# Data 확인
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44078 entries, 0 to 44077
Columns: 113 entries, lot_id to sa4_box_dan
dtypes: float64(10), int64(54), object(49)
memory usage: 38.0+ MB


In [5]:
# Data 확인
print("Data 구조:", df_raw.shape)
print()
print("변수 : ", df_raw.columns)

Data 구조: (44078, 113)

변수 :  Index(['lot_id', 'tray_id', 'dt_start', 'cell_id', 'judge', 'rta1_cell_no',
       'rta1_box_col', 'rta1_box_row', 'rta1_box_dan', 'hta1_box_col',
       ...
       'ocv1_deltaocv', 'm1_thick', 'm1_voltage', 'm1_res_ac', 'm1_mv',
       'm1_voltage_an', 'dt_start_old', 'sa4_box_row', 'sa4_box_col',
       'sa4_box_dan'],
      dtype='object', length=113)


### 1-1. 필요한 열만 선택

In [6]:
# 모델에 필요한 열을 아래에 작성
columns_to_keep = ['judge','tray_id',
                   'c1_temp_avg','c2_temp_avg','c3_temp_avg','c4_temp_avg',
                   'dc1_temp_avg','dc2_temp_avg','dc3_temp_avg'
                    ] # 필요한 열을 모두 나열

# 선택한 열로 데이터프레임을 필터링
df_filtered = df_raw[columns_to_keep]


In [7]:
df_filtered.head()

,judge,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg
0,양품,TRAY-1023,268,265,266,273,266,266,269
1,양품,TRAY-1023,268,265,266,273,266,266,269
2,양품,TRAY-1023,268,265,266,273,266,266,269
3,양품,TRAY-1023,268,265,266,273,266,266,269
4,양품,TRAY-1023,268,265,266,273,266,266,269


### 1-2. 결측치 제거

In [8]:
# 결측치 확인
missing_values = df_filtered.isnull().sum(axis = 1)

In [9]:
# 결측치가 특정 임계값보다 큰 행을 삭제
threshold = 1  # 예를 들어, 결측치가 1개 이상인 행을 삭제
rows_to_delete = missing_values[missing_values > threshold].index

# 97개 행 삭제
df_cleaned = df_filtered.drop(index=rows_to_delete)

In [10]:
# 97개 행 삭제 확인
df_cleaned

,judge,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg
0,양품,TRAY-1023,268,265,266,273,266,266,269
1,양품,TRAY-1023,268,265,266,273,266,266,269
2,양품,TRAY-1023,268,265,266,273,266,266,269
3,양품,TRAY-1023,268,265,266,273,266,266,269
4,양품,TRAY-1023,268,265,266,273,266,266,269
...,...,...,...,...,...,...,...,...,...
44073,양품,TRAY-2791,255,256,255,255,256,256,255
44074,양품,TRAY-2791,255,256,255,255,256,256,255
44075,양품,TRAY-2791,255,256,255,255,256,256,255
44076,양품,TRAY-2791,255,256,255,255,256,256,255


### 1-3. judge 더미화

In [11]:
# 'judge' 열 더미화 (불량을 1로, 양품을 0으로)
df_raw = pd.get_dummies(df_cleaned, columns = ["judge"])

df_raw

,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg,judge_불량,judge_양품
0,TRAY-1023,268,265,266,273,266,266,269,0,1
1,TRAY-1023,268,265,266,273,266,266,269,0,1
2,TRAY-1023,268,265,266,273,266,266,269,0,1
3,TRAY-1023,268,265,266,273,266,266,269,0,1
4,TRAY-1023,268,265,266,273,266,266,269,0,1
...,...,...,...,...,...,...,...,...,...,...
44073,TRAY-2791,255,256,255,255,256,256,255,0,1
44074,TRAY-2791,255,256,255,255,256,256,255,0,1
44075,TRAY-2791,255,256,255,255,256,256,255,0,1
44076,TRAY-2791,255,256,255,255,256,256,255,0,1


In [12]:
# judge 열로 만들기 : 불량은 1이고, 양품은 0으로 변환됨
df_raw["judge"] = df_raw["judge_양품"] + df_raw["judge_불량"]*2 -1

df_raw

,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg,judge_불량,judge_양품,judge
0,TRAY-1023,268,265,266,273,266,266,269,0,1,0
1,TRAY-1023,268,265,266,273,266,266,269,0,1,0
2,TRAY-1023,268,265,266,273,266,266,269,0,1,0
3,TRAY-1023,268,265,266,273,266,266,269,0,1,0
4,TRAY-1023,268,265,266,273,266,266,269,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
44073,TRAY-2791,255,256,255,255,256,256,255,0,1,0
44074,TRAY-2791,255,256,255,255,256,256,255,0,1,0
44075,TRAY-2791,255,256,255,255,256,256,255,0,1,0
44076,TRAY-2791,255,256,255,255,256,256,255,0,1,0


In [13]:
df_raw["judge"] = df_raw["judge"].astype("int64")

In [14]:
# 필요없는 변수 제거
df_raw = df_raw.drop(["judge_양품","judge_불량"], axis=1)

df_raw

,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg,judge
0,TRAY-1023,268,265,266,273,266,266,269,0
1,TRAY-1023,268,265,266,273,266,266,269,0
2,TRAY-1023,268,265,266,273,266,266,269,0
3,TRAY-1023,268,265,266,273,266,266,269,0
4,TRAY-1023,268,265,266,273,266,266,269,0
...,...,...,...,...,...,...,...,...,...
44073,TRAY-2791,255,256,255,255,256,256,255,0
44074,TRAY-2791,255,256,255,255,256,256,255,0
44075,TRAY-2791,255,256,255,255,256,256,255,0
44076,TRAY-2791,255,256,255,255,256,256,255,0


# 2. tray별 불량율 계산

### 2-1. tray별 불량 개수

In [15]:
# df_raw에서 tray_id와 judge 열만 선택합니다.
subset_df = df_raw[['tray_id', 'judge']]

# tray_id로 그룹화하고 각 그룹 내의 judge 값을 합산합니다.
result_df = subset_df.groupby('tray_id')['judge'].sum().reset_index()

# 결과를 출력합니다.
print(result_df)

        tray_id  judge
0     TRAY-1002      0
1     TRAY-1003      0
2     TRAY-1004      0
3     TRAY-1005      0
4     TRAY-1006      0
...         ...    ...
1784  TRAY-2787      4
1785  TRAY-2788      6
1786  TRAY-2789      3
1787  TRAY-2790      1
1788  TRAY-2791      5

[1789 rows x 2 columns]


### 2-2. tray의 cell 개수 세기

In [16]:
import pandas as pd

# 데이터프레임에서 tray_id 열 중에서 같은 값을 가진 행의 개수를 구합니다.
tray_id_counts = df_raw['tray_id'].value_counts().reset_index()
tray_id_counts.columns = ['tray_id', 'count']

# 결과를 출력합니다.
print(tray_id_counts)

        tray_id  count
0     TRAY-1901     25
1     TRAY-2213     25
2     TRAY-2171     25
3     TRAY-2165     25
4     TRAY-2164     25
...         ...    ...
1784  TRAY-1518      4
1785  TRAY-2776      2
1786  TRAY-2778      1
1787  TRAY-2767      1
1788  TRAY-2769      1

[1789 rows x 2 columns]


### 2-3. tray별 불량률 계산

In [17]:
import pandas as pd

# 데이터프레임에서 tray_id 열 중에서 같은 값을 가진 행의 개수를 구합니다.
tray_id_counts = df_raw['tray_id'].value_counts().reset_index()
tray_id_counts.columns = ['tray_id', 'count']

# tray_id 별로 judge 값을 합산한 결과 데이터프레임
subset_df = df_raw[['tray_id', 'judge']]
result_df = subset_df.groupby('tray_id')['judge'].sum().reset_index()
result_df.columns = ['tray_id', 'judge_sum']

# tray_id_counts와 result_df를 병합하여 비율을 계산합니다.
merged_df = tray_id_counts.merge(result_df, on='tray_id', how='left')

# count와 judge_sum을 사용하여 비율을 계산합니다.
merged_df['ratio'] = merged_df['judge_sum'] / merged_df['count'] * 100

# 결과를 출력합니다.
print(merged_df)


        tray_id  count  judge_sum  ratio
0     TRAY-1901     25          0    0.0
1     TRAY-2213     25          1    4.0
2     TRAY-2171     25         12   48.0
3     TRAY-2165     25          0    0.0
4     TRAY-2164     25          0    0.0
...         ...    ...        ...    ...
1784  TRAY-1518      4          0    0.0
1785  TRAY-2776      2          0    0.0
1786  TRAY-2778      1          0    0.0
1787  TRAY-2767      1          0    0.0
1788  TRAY-2769      1          0    0.0

[1789 rows x 4 columns]


In [18]:
# df_raw 데이터프레임에 merged_df의 'tray_id'와 'ratio' 열을 병합합니다.
df_raw = df_raw.merge(merged_df[['tray_id', 'ratio']], on='tray_id', how='left')

# 결과를 출력합니다.
print(df_raw)

# 변경된 데이터를 새로운 CSV 파일로 저장 (경로 지정)
df_raw.to_csv('/home/piai/바탕화면/전처리_data.csv', index=False)

         tray_id  c1_temp_avg  c2_temp_avg  c3_temp_avg  c4_temp_avg  \
0      TRAY-1023          268          265          266          273   
1      TRAY-1023          268          265          266          273   
2      TRAY-1023          268          265          266          273   
3      TRAY-1023          268          265          266          273   
4      TRAY-1023          268          265          266          273   
...          ...          ...          ...          ...          ...   
44073  TRAY-2791          255          256          255          255   
44074  TRAY-2791          255          256          255          255   
44075  TRAY-2791          255          256          255          255   
44076  TRAY-2791          255          256          255          255   
44077  TRAY-2791          255          256          255          255   

       dc1_temp_avg  dc2_temp_avg  dc3_temp_avg  judge  ratio  
0               266           266           269      0    0.0  
1      

# 3. XGBoost
'judge','tray_id',
'c1_temp_avg','c2_temp_avg','c3_temp_avg','c4_temp_avg',
'dc1_temp_avg','dc2_temp_avg','dc3_temp_avg',
                   'c1_voltage_avg','c2_voltage_avg','c3_voltage_avg','c4_voltage_avg',
'dc1_voltage_avg','dc2_voltage_avg','dc3_voltage_avg'

### 3-1. XGBoost의 성능

In [19]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv("/home/piai/바탕화면/전처리_data.csv")

# 독립 변수와 종속 변수 설정
X = df[['c1_temp_avg', 'c2_temp_avg', 'dc1_temp_avg', 'dc2_temp_avg']]  # 독립 변수
y = df['judge']  # 종속 변수 (불량 여부)

# 데이터 분할 (학습용, 테스트용)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# XGBoost 분류 모델 생성
model = XGBClassifier(learning_rate=0.2, n_estimators=200)

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 오차 행렬
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# 정확도
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# 정밀도
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# 재현율
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# F1-score
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")

# AUC
# XGBoost의 경우 predict_proba를 사용하여 확률을 얻어야 합니다.
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")


Confusion Matrix:
[[12752    33]
 [  398    41]]
Accuracy: 0.9674077434966727
Precision: 0.5540540540540541
Recall: 0.09339407744874716
F1-score: 0.1598440545808967
AUC: 0.8785501054321381


### 3-2. XGBoost로 judge(0,1)예측

In [20]:
import pandas as pd
from xgboost import XGBClassifier

# 데이터프레임을 불러옵니다. (예시: data.csv 파일을 읽어옴)
df = pd.read_csv("/home/piai/바탕화면/전처리_data.csv")

# 적정 온도 범위 설정
lower_bound = 25.65
upper_bound = 29.75

# 적정 온도 범위를 벗어나는 행 찾기
out_of_range = df['dc2_temp_avg'].loc[(df['dc2_temp_avg'] < lower_bound) | (df['dc2_temp_avg'] > upper_bound)]

# 적정 온도로 값 대체 (평균값으로)
df.loc[out_of_range.index, 'dc2_temp_avg'] = (lower_bound + upper_bound) / 2

# 독립 변수(X)와 종속 변수(y)를 지정합니다.
X = df[['c1_temp_avg', 'c2_temp_avg', 'dc1_temp_avg', 'dc2_temp_avg']]  # 독립 변수
y = df['judge']  # 종속 변수 (불량 여부)

# XGBoost 분류 모델을 생성하고 학습합니다.
model = XGBClassifier(learning_rate=0.2, n_estimators=200)
model.fit(X, y)

# 예측값을 추출합니다.
predictions = model.predict(X)

# 예측값을 데이터프레임에 추가합니다.
df['predicted_judge'] = predictions

df.head()


,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg,judge,ratio,predicted_judge
0,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
1,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
2,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
3,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
4,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0


In [21]:
# CSV 파일로 저장합니다.
df.to_csv('/home/piai/바탕화면/predicted_data.csv', index=False)

### 3-3. 적정 온도로 높인 후 불량율 예측

In [22]:
df = pd.read_csv("/home/piai/바탕화면/predicted_data.csv")
df.head()

,tray_id,c1_temp_avg,c2_temp_avg,c3_temp_avg,c4_temp_avg,dc1_temp_avg,dc2_temp_avg,dc3_temp_avg,judge,ratio,predicted_judge
0,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
1,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
2,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
3,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0
4,TRAY-1023,268,265,266,273,266,27.7,269,0,0.0,0


In [23]:
# 데이터프레임에서 'predicted_judge' 열의 숫자 합계 계산
predicted_judge_sum = df['predicted_judge'].sum()
print(f"predicted_judge 열의 숫자 합계: {predicted_judge_sum}")

# 데이터프레임의 크기 확인
num_rows = df_raw.shape[0]  # 행의 개수만 추출
print(f"데이터프레임의 행의 개수: {num_rows}")

predicted_judge 열의 숫자 합계: 200
데이터프레임의 행의 개수: 44078


In [25]:
predicted_ratio = (predicted_judge_sum / num_rows) * 100
rounded_ratio = round(predicted_ratio, 3)  # 숫자를 소수점 세 자리까지 반올림
print(f"적정 온도로 바꾼 후 예측한 불량율 : {rounded_ratio}%")

적정 온도로 바꾼 후 예측한 불량율 : 0.454%
